In [1]:
import sys, os

# ✅ force Jupyter to recognize smartqc module
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

print(f"✅ Project root added: {project_root}")

✅ Project root added: C:\Users\Admin\smart_qc_v2


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), ".."))) #first priority in project root 
from smartqc.config import CONFIG
print(CONFIG)

{'batch_size': 100000, 'random_state': 42, 'verbose': True, 'anomaly_detection': True, 'inf_handling': 'knn', 'columns_to_drop': [], 'scale_numeric': True, 'use_smote': True, 'smote_sampling_strategy': 0.5, 'auto_target_selection': False, 'target_column_names': [], 'time_series_threshold': 0.3, 'max_trials': 10, 'automl_timeout_minutes': 5}


In [4]:
from smartqc.data_loader import DataLoader

data_loader = DataLoader("../data/ai4i2020.csv")
df = data_loader.df
print(df.shape)
data_loader.preview()

Data loaded → shape: (10000, 14)
(10000, 14)
   UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0    1     M14860    M                298.1                    308.6   
1    2     L47181    L                298.2                    308.7   
2    3     L47182    L                298.1                    308.5   
3    4     L47183    L                298.2                    308.6   
4    5     L47184    L                298.2                    308.7   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Machine failure  TWF  \
0                    1551         42.8                0                0    0   
1                    1408         46.3                3                0    0   
2                    1498         49.4                5                0    0   
3                    1433         39.5                7                0    0   
4                    1408         40.0                9                0    0   

   HDF  PWF  OSF  RNF  
0    0    0

In [5]:
from smartqc.missing_handler import MissingHandler

mh = MissingHandler(data_loader.df)
clean_df = mh.fix()
print(clean_df.shape)

 Missing Value Audit:
   Dataset Shape: (10000, 14)
   Total Missing Cells: 0
   Missing %: 0.00%
Using KNNImputer to fill missing values...
KNNImputer complete. Non-numeric columns untouched: ['Product ID', 'Type']
New dataset shape after missing value handling: (10000, 14)
(10000, 14)


In [6]:
from smartqc.quality_monitor import get_user_sensor_limits, QualityMonitor

manual_ranges = get_user_sensor_limits(clean_df)  # ← system asks user Y/N
qm = QualityMonitor(clean_df, manual_outlier_ranges=manual_ranges)
audited_df = qm.get_data()


Do you want to manually define sensor limits? (y/n):  n


Skipping manual outlier detection. Will proceed with automatic (if enabled).
Data Quality Report:
   Dataset Shape: (10000, 14)
   Infinite values detected: 0
Running automatic anomaly detection (Isolation Forest)...
Automatic outliers flagged in 'is_outlier' column (1=outlier, 0=normal).
   is_outlier  UDI  Air temperature [K]  Process temperature [K]  \
0           0  1.0                298.1                    308.6   
1           0  2.0                298.2                    308.7   
2           0  3.0                298.1                    308.5   
3           0  4.0                298.2                    308.6   
4           0  5.0                298.2                    308.7   

   Rotational speed [rpm]  Torque [Nm]  
0                  1551.0         42.8  
1                  1408.0         46.3  
2                  1498.0         49.4  
3                  1433.0         39.5  
4                  1408.0         40.0  


In [ ]:
from smartqc.defect_engineer import DefectEngineer

de = DefectEngineer(audited_df)   # audited_df from Step 3 (quality monitor)
final_df, selected_targets = de.run()

Select your target column(s) for prediction:


SelectMultiple(description='Targets:', layout=Layout(width='50%'), options=('UDI', 'Product ID', 'Type', 'Air …

Button(description='Confirm Selection', style=ButtonStyle())

Output()

Waiting for user to select target(s)...
